# Import et installation des librairies

In [ ]:
%pip install plotly polars
%pip install nbformat
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import polars as pl

# Import du CSV

In [ ]:
df = pl.read_csv("Données_sommeil.csv", separator=",")

In [ ]:
df.describe()

In [ ]:
df.head()

# Visualisation

## Dates

In [ ]:
fig3 = px.histogram(df, x="Day")
fig3.show()

On voit bien une distribution plutot uniforme des dates, il n'y a donc pas de problématique de saisonnalité ou autre, la date n'est donc pas une donnée à prendre en compte.

## Temps au lit vs temp de sommeil

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df["Time in Bed"], name="Minutes au lit", marker_color="red"))
# fig.add_trace(go.Histogram(x=df["Minutes REM Sleep"], name="Sommeil REM", marker_color="purple"))
fig.add_trace(go.Histogram(x=df["Minutes Deep Sleep"], name="Sommeil profond", marker_color="blue"))
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2)

trace1 = go.Histogram(x=df["Time in Bed"], name="Temps au lit", marker_color="black")
trace2 = go.Histogram(x=df["Minutes Light Sleep"], name="Sommeil profond", marker_color="red")
trace3 = go.Histogram(x=df["Minutes REM Sleep"], name="Sommeil REM", marker_color="purple")
trace4 = go.Histogram(x=df["Minutes Deep Sleep"], name="Sommeil profond", marker_color="blue")

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig.update_layout(title="Histogrammes des différents temps passés au lit et des types de sommeil")

for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text='Minutes', row=i, col=j)
        fig.update_yaxes(title_text='Effectif', row=i, col=j)

fig.show()

## Niveau d'activité

In [ ]:
			
fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Box(x=df["Minutes Sedentary"], name="Temps sédentaire", marker_color="black"), row=1, col=1)
fig.add_trace(go.Box(x=df["Minutes Lightly Active"], name="Temps d'activité faible", marker_color="green"), row=2, col=1)
fig.add_trace(go.Box(x=df["Minutes Fairly Active"], name="Temps d'activité modérée", marker_color="yellow"), row=3, col=1)
fig.add_trace(go.Box(x=df["Minutes Very Active"], name="Temps d'activité forte", marker_color="red"), row=4, col=1)

fig.update_layout(title="Boites à moustaches des différents temps de sédentarité et d'activité en minutes")

fig.show()

Rajouter des graphiques par rapport aux pas et aux calories... à voir


In [ ]:
fig = px.density_heatmap(df, x="Steps", y="Calories Burned")
fig.show()

In [ ]:
fig = px.scatter(df, x="Steps", y="Calories Burned")
fig.show()

In [ ]:
bin_edges = [10000, 15000, 20000]
steps_bin = pl.Series(df["Steps"].cut(breaks=bin_edges, labels=["<=10k", "<=15k", "<=20k", ">20k"]))

In [ ]:
df_bis =  df.with_columns(steps_bin.alias("Step Brackets"))

In [ ]:
df_bis.head(3)

In [ ]:
fig = px.scatter_3d(df_bis, x="Step Brackets", y="Time in Bed", z="Minutes Awake")
fig.show()

In [ ]:
fig = px.scatter(df_bis, x="Step Brackets", y="Minutes Awake")
fig.show()

In [ ]:
df_bis = df_bis.with_columns((pl.col("Minutes Awake")/pl.col("Time in Bed")*100)).alias("Awake Ratio")

In [ ]:
df_bis = df_bis.with_columns(
    pl.col("Minutes Awake").cut(
        breaks=[10, 20, 30, 40, 50],
        labels=["<=10%", "<=20%", "<=30%", "<=40k%", "<=50%", ">50%"]).alias("Awake Ratio Brackets"))

In [ ]:
px.scatter(df_bis, x="Step Brackets", y="Awake Ratio").show()